In [1]:
#####Reccurent Unit Latent Factor model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
import gc
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id
    if pattern==1:
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z

In [3]:
####データの生成####
##データの設定
k = 10
hh = 5000   
item = 3000
pt = np.random.poisson(np.random.gamma(20, 1/0.225, hh))
N = np.sum(pt)
m = np.random.poisson(np.random.gamma(10, 1/0.3, N)); m[m <= 2] = 2
M = np.sum(m)

In [4]:
##idとインデックスの設定
#idの設定
d_id = np.repeat(np.arange(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))
m_id = np.repeat(np.arange(N), m)

#インデックスの設定
freq_m = np.repeat(0, hh)
d_index = [i for i in range(hh)]
m_index = [i for i in range(N)]
for i in range(hh):
    d_index[i] = np.array(np.where(d_id==i)[0], dtype="int")
    freq_m[i] = np.sum(m[d_index[i]])
for i in range(N):
    if i==0:
        m_index[i] = np.arange(m[0])
    else:
        m_index[i] = np.max(m_index[i-1]) + np.arange(m[i]) + 1
freq_id = np.repeat(np.arange(hh), freq_m)

In [5]:
##アイテムの生成
#多項分布からトピックを生成
topic = 25
theta_topic = np.random.dirichlet(np.repeat(0.2, topic), hh)
phi_topic = np.random.dirichlet(np.repeat(0.2, item), topic)
z = np.array(rmnom(theta_topic[np.repeat(d_id, m), ], M, topic, np.arange(M), 0), dtype="int")

#トピックからアイテムを生成
item_id = np.repeat(0, M)
item_dt = np.full((N, np.max(m)), item, dtype="int16")
for i in range(N):
    if i%10000==0:
        print(i)
    index = m_index[i]
    target_z = z[index]
    temp_id = np.array(rmnom(phi_topic[target_z, ], m[i], item, np.arange(m[i]), 0), dtype="int16")
    item_id[index] = temp_id
    item_dt[i, np.arange(m[i])] = temp_id

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000


In [119]:
##応答変数を生成
#階層モデルのパラメータを生成


array([[2646, 2391,  887, ..., 3000, 3000, 3000],
       [1389,  160, 1169, ..., 3000, 3000, 3000],
       [1275,  334,  855, ..., 3000, 3000, 3000],
       ...,
       [2675, 1655,  579, ..., 3000, 3000, 3000],
       [ 695, 1495, 1703, ..., 3000, 3000, 3000],
       [ 443, 2478,   11, ..., 3000, 3000, 3000]], dtype=int16)